In [2]:
import sagemaker
import boto3
# boto_session=boto3.Session(region_name='us-east-1')
sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20231030T210397')['Role']['Arn']
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name ravi_tej to get Role path.


sagemaker role arn: arn:aws:iam::005418323977:role/service-role/AmazonSageMaker-ExecutionRole-20231030T210397
sagemaker bucket: sagemaker-ap-south-1-005418323977
sagemaker session region: ap-south-1


In [3]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  version="1.3.3"
)

In [10]:
import json

In [11]:
model_id = 'NousResearch/Nous-Hermes-2-SOLAR-10.7B'

In [77]:
instance_type = "ml.g5.4xlarge"
number_of_gpu = 1
health_check_timeout = 600
quantize="gptq"
num_shard=2
bits=4
group_size=128
revision = 'gptq-8bit-32g-actorder_True'

In [1]:
config = {
    'HF_MODEL_ID':model_id,
    'SM_NUM_GPUS': json.dumps(number_of_gpu),
    # 'WORLD_SIZE': 2
    # 'QUANTIZE':  quantize,
    'SHARDED': json.dumps(True),
    'NUM_SHARD': json.dumps(2),
    # 'REVISION': revision,
    'MAX_INPUT_LENGTH': json.dumps(2560), # Max length of input text
    'MAX_TOTAL_TOKENS': json.dumps(3072), # Max length of the generation (including input text)
    'MAX_BATCH_PREFILL_TOKENS': json.dumps(8144), # Max length of the generation (including input text)
    'MAX_BATCH_TOTAL_TOKENS': json.dumps(8144),
    # 'GPTQ_BITS': json.dumps(bits),
    # 'GPTQ_GROUPSIZE': json.dumps(group_size),
    'DISABLE_CUSTOM_KERNELS': json.dumps(True)
}

NameError: name 'model_id' is not defined

In [79]:
from sagemaker.huggingface.model import HuggingFaceModel

In [80]:
yi_model = HuggingFaceModel(role=role, image_uri=llm_image, env=config, sagemaker_session=sess)

In [84]:
# Deploy model to an endpoint
# https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy
llm = yi_model.deploy(
    endpoint_name = 'SOLAR-TGI-2',
  initial_instance_count=2,
  instance_type=instance_type,
  container_startup_health_check_timeout=health_check_timeout, # 10 minutes to be able to load the model
)

---------------------------*

UnexpectedStatusException: Error hosting endpoint SOLAR-TGI-2: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.. Try changing the instance type or reference the troubleshooting page https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-troubleshooting.html

In [22]:
endpoint_name

'SOLAR-TGI'

In [44]:
parameters = {
    "do_sample": True,
    "top_p": 0.9,
    "temperature": 0.8,
    "max_new_tokens": 512,
    "repetition_penalty": 1.2,
    "stop": ["</s>"],
    'return_full_text': False
}

In [45]:
smr = sess.boto_session.client("sagemaker-runtime")

In [127]:
# def generate_template(prompt):
#     return f'''USER: {prompt}
#     ASSISTANT:
#     '''

# len(prompt.split(' '))

In [46]:
from datetime import datetime

In [47]:
system_prompt = '''|instructions_start|
You're an AI expert in financial and business topics, answering queries using relevant materials and following strict rules:
* Valid Queries: Questions about finance, business, or financially relevant entities (e.g., India, RBI, Reliance, Fed).
* Invalid Queries: Questions unrelated to finance or business (e.g., Bollywood, sports, politics unless financially relevant), instructional queries on non-financial topics (jokes, coding, recipes), and inappropriate or harmful content.

Response Instructions:
* If the query is valid and answerable with references, summarize the information clearly and concisely.
* If the query is an entity like India, Supreme Court, Modi, RBI - and if the articles contain news about these entities - then you can mention that the latest news about ‘query’ by summarising the articles. When the published date mentioned in the content is slightly older - you can mention the corresponding dates as well. If it is new, then no need to do that. 
* If the references have information related to both India and international entities, then first prefer highlighting the indian entities. When referring to international, explicitly mention the country, since indian readers shouldn’t get confused
* If it's a valid query but the exact answer isn't in the references, state your inability to find the exact answer and provide a summary of related information.
* If it's a valid query but there's no relevant information, apologize, say you can't find the answer, and note that your team is working to expand knowledge.
* For invalid queries, politely decline, noting your focus on finance, and optionally share brief financial trivia.

General Response Guidelines:
* Address users professionally and respectfully.
* Ensure good grammar and readability.
* Keep responses concise, typically 3-4 sentences, maximum 6.
* Don't provide answers not explicitly found in the references.
* If an exact answer isn't found, then mention that you are not able to find the answer and follow with a concise summary of any related information available in the references.
* Politely decline irrelevant queries, emphasizing your financial focus.
* For queries about recent dates, calculate the period using today's date in the format %Y-%m-%d
* Do not mention ‘OpenAI’ at any cost. You are simply a financial AI assistant designed by WintWealth.

The response includes the query and references labeled accordingly. The user's query is mentioned below between user_query and |query_end|
'''

In [48]:
def create_prompt_for_query_references(search_query, references):
    today_date_string = f"today's date: {datetime.now().strftime('%Y-%m-%d')}"
    query_string = f"user_query: {search_query} |query_end| "
    references_string = ''
    for i in range(len(references)):
        references_string += f'reference {i}: {references[i]} \n\n'
    return system_prompt + today_date_string + '|instructions_end| ' + query_string + references_string

In [49]:
ref = ['''
title: What is XIRR in Mutual Funds: Calculation & Formula in Excel. published_date: 2023-02-13T11:50:00+00:00. content: XIRR or Extended Internal Rate of Return helps calculate the return on mutual fund investments made through SIP (Systematic Investment Plan). CEO of Navi Mutual Fund, Hari Shyamsunder says, “Your investments should always be compared against a possible alternative. But to make that comparison, you must understand the basic concept of XIRR.” This article is a beginner’s guide to understanding the XIRR meaning in mutual funds, its importance and the calculation of your SIP returns. Read on! What is XIRR in Mutual Funds? XIRR Meaning – XIRR (Extended Internal Rate of Return) can be defined as a computation method for calculating returns of mutual fund investments at irregular intervals. It can also be defined as a single rate of return which can be applicable for every instalment and redemption. You can use it to have an understanding of the current value of all your investments. When you invest in a scheme via SIP (Systematic Investment Plan) , you need to pay multiple instalments. You can easily calculate the returns of these investments with the help of IRR (Internal Rate of Return), but, you need to make sure that the time frame between the consecutive cash flows remains the same.The XIRR method simplifies the entire process of calculating returns even in the case of irregular investments. You can simply use an excel sheet to calculate your investment returns. Here, the XIRR formula will modify the IRR and enable you to allocate separate dates to individual cash flows. XIRR Formula and Calculation To calculate XIRR in mutual funds, you have to insert the transactions (additional purchases, SIP/SWP instalments, redemption) and the corresponding dates in the designated place in an MS Excel sheet. These transaction details will be available in the statement of account sent by the AMC (Asset Management Company) . To calculate a mutual fund’s XIRR, you can use the XIRR formula in MS Excel: “= XIRR (values, dates, guess)” Here, you have to insert cash outflows (lump-sum purchases and SIP instalments) as negative values (i.e. put a minus sign before the amount) and cash inflows (dividends, SWP, redemptions) as positive values. ‘Guess’ is an optional input. It is considered to be 0.1 by default. If you have not redeemed your mutual fund units, then, to calculate XIRR, you have to enter the current investment value with the NAV (Net Asset Value) of your mutual fund investment. Do not include transactions like dividend reinvestment because they do not involve actual cash flows. Moreover, if you are calculating the XIRR at the level of schemes, then switches should be treated as redemption. But, if you are calculating XIRR in mutual funds as portfolio level, then switches are not relevant in the calculation. Also Read How to Use SIP CAGR Calculator to Estimate Returns on Mutual Fund Investments? Best Low-Risk Mutual Funds Example of XIRR Calculation in Excel Here is an example of a six-month SIP to calculate XIRR in mutual funds: SIP amount = Rs.5,000 SIP investment dates: Start on 01-01-2017, End on 01-06-2017 Date of redemption: 01-07-2017 Maturity sum = Rs.31,000 Let’s assume cash flows as mentioned in the table below: 01/01/2017 -5000 03/02/2017 -5000 01/03/2017 -5000 11/04/2017 -5000 01/05/2017 -5000 25/06/2017 -5000 01-07-2017 31000 XIRR 11.92429 Let’s take a look at the calculation part below: In the left column, all the dates are mentioned. The right column includes your SIP instalments of Rs.5,000. It is marked as negative owing to the outflow of cash. In the left column, you can find the redemption date. Just beside that, there is a redemption amount of Rs.31,000. Use the XIRR formula: Insert the range of cells that represent the transaction values. After that, enter the series of payment dates with respect to the value of the cash flows. You will have your XIRR value of 11.92 %, as mentioned in the table. Why is XIRR Accurate for Mutual Funds? Let us assume that you had started investing in a scheme via a monthly SIP of Rs.15,000 and then continued the investment for 5 years. Supposed, after 5 years, the value of the investment grew to Rs.12 lakh. Here, the annual return of the first month’s instalment will be different from the rest as it had remained invested for the maximum number of months. In simpler words, the CAGR of each instalment will differ as their investment period will be different. Analysing the performance of a mutual fund will become complicated when you look at the CAGRs of these instalments. To make the task easier, all the CAGRs are adjusted together to a common CAGR which is then considered as a mutual fund’s XIRR. XIRR vs CAGR – Can we use CAGR Instead of XIRR? This is a common misconception that CAGR and XIRR are the same. CAGR is the accurate calculation of returns on every instalment, which doesn’t take inflows and outflows into consideration. In contrast, XIRR means the overall average of the CAGRs and takes into account every inflow & outflow of cash. Here are some simplified pointers to make their differences obvious: XIRR CAGR XIRR meaning is an average annualised return. CAGR is the absolute annualised return. XIRR takes into account every cash inflow and outflow. CAGR is calculated only considering the initial value, absolute value and investment tenure. XIRR is calculated when there are multiple cash flows; hence, it can be good for SIP investments. CAGR is ideal for lump-sum investments . XIRR means an average of every return earned by each cash flow during tenure. CAGR is the compounded growth rate. Final Word XIRR is the most comprehensive way of determining your returns in case of multiple transactions, especially when your investment time is irregular. In other words, allocating your money to mutual funds via an SIP will lead to cash outflows and cash inflows. In such cases, the time of investment also plays a significant role in the calculation of the returns. To that end, the role of Extended Internal Rate of Return (XIRR) in SIP becomes essential. If you’re looking to explore diverse low-cost funds, check out Navi Mutual Fund . Download the Navi app to start SIP at just Rs.10 per month. Disclaimer: Mutual Fund investments are subject to market risks, read all scheme related documents carefully. FAQs What is the formula of XIRR? If you wish to calculate XIRR in mutual funds, you need to enter all transactions and dates in corresponding columns in an MS Excel sheet. Then, you have to enter the following formula: “= XIRR (values, dates, guess). Cash outflows have to be entered as negative outflows and cash inflows as positive. What is the formula of CAGR? The formula for CAGR is as follows: CAGR ={(Ending balance/beginning balance)^1/n} – 1 Where, beginning balance = Investment value at the beginning of the investment period Ending balance = Investment value at the end of your investment N = Total number of years you have invested in a mutual fund What is IRR? IRR (Internal Rate of Return) is described as a discounting cash flow technique which provides the rate of return that a mutual fund has earned. Here, both the discounted cash inflow and the total initial cash outlay are equal to zero. Is XIRR better than IRR? IRR is used to determine returns against investments made at fixed intervals. But investments and redemptions are never regular. In that case, XIRR is the right choice. What is SIP in mutual funds? A SIP or Systematic Invest Plan is a tool to invest in mutual funds. It is a systematic way to invest a fixed sum of money in a periodical manner. It is also a very safe way to invest. What is XIRR in NPS or National Pension Scheme? Return on Investments for National Pension Scheme gives the annualised effective compounded return rate in the PRAN account and is calculated using the formula of XIRR. The calculation is done considering all the contribution/redemptions processed in the PRAN account since inception and the latest valuation of the investments. What is XIRR Return? XIRR Return is the returns on your mutual fund investments made at irregular intervals, like SIP investments. XIRR calculates the CAGR of every SIP investment made. These are then added together to give you the overall CAGR. Which is better – CAGR or XIRR? CAGR gives you the value of your annual return over a period of time longer than one year. But in the case of calculating returns on frequent investments or cash flows during different time periods, such a method is not apt as it does not consider multiple cash flows. In such cases, XIRR is an ideal option. Does the XIRR function work monthly in Excel? The XIRR function in MS Excel always computes an annualised IRR even if an individual calculates returns on weekly/monthly cash flows. What is XIRR Full Form? XIRR stands for Extended Internal Rate of Return. Before you go… Want to put your savings into action and kick-start your investment journey 💸 But don’t have time to do research? Invest now with Navi Nifty 50 Index Fund , sit back, and earn from the top 50 companies. Disclaimer: Mutual Fund investments are subject to market risks, read all scheme-related documents carefully. This article has been prepared on the basis of internal data, publicly available information and other sources believed to be reliable. The information contained in this article is for general purposes only and not a complete disclosure of every material fact. It should not be construed as investment advice to any party. The article does not warrant the completeness or accuracy of the information and disclaims all liabilities, losses and damages arising out of the use of this information. Readers shall be fully liable/responsible for any decision taken on the basis of this article. Latest Webstories Update your Aadhaar card details online for free before June 14, 2023 30 March 2023 6 Investment Tips From Raamdeo Agarwal You Cannot Ignore! 7 Creative Home Decor Ideas to Transform Your Space Finance Bill 2023 Highlights A Tribute to Stock Market Expert: Ashwani Gujral Ram Navami 2023: 6 Life Lessons to Learn From Lord Rama 23 March 2023 11 Facts You Didn’t Know About the Man Behind Chaptgpt: Sam Altman 1 March 2023 6 Best Nifty Next 50 Funds to Invest in 2023 27 February 2023 7 Best Investment Apps in India 2023 A HERtorical Investment: The Mahila Samman Savings Certificate
''']

In [56]:
request = {"inputs":'WHAT IS XIRR', "parameters": parameters, "stream": False}

In [57]:
resp = smr.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(request),
    ContentType="application/json",
)

In [58]:
resp['Body'].read()

b'[{"generated_text":"?\\nXirr stands for \\"Excel Internal Rate of Return\\". This is a calculation used to determine the average rate of return earned on an investment. It\'s most commonly found in Excel, but there are other programs that use it as well such as Google Sheets and OpenOffice Calc.\\nThe term was coined by Microsoft when they introduced their spreadsheet software called \\"Microsoft Office Excel\\" back in 1985. Since then, many other applications have adopted this feature under different names like IRR (Internal Rate Of Return) or NPV (Net Present Value). However, regardless of what you call it, its purpose remains unchanged: determining the expected future value given current cash flows and discount rates.\\nIn essence, xirr calculates how much money would be needed today to equal tomorrow\'s expected earnings after accounting for taxes, inflation, interest payments etc. In simpler terms, if you invest $100 today at a rate of 12%, your total amount after one year will

In [373]:
# resp = smr.invoke_endpoint(
#     EndpointName='OpenHermes-ApricotTangerineScallop-TGI',
#     Body=json.dumps(request),
#     ContentType="application/json",
# )

In [381]:
a = resp['Body'].read()

In [382]:
a

b'[{"generated_text":"https://www.taxguru.co.in/articles/section-80c-understanding-deductions-under-section-80c.html</s>"}]'

### Deleting the endpoint

In [59]:
sm_client = boto3.client('sagemaker')

In [82]:
endpoint_name = 'SOLAR-TGI-2'
endpoint = sm_client.describe_endpoint(EndpointName=endpoint_name)
endpoint_config_name = endpoint['EndpointConfigName']

ClientError: An error occurred (ValidationException) when calling the DescribeEndpoint operation: Could not find endpoint "SOLAR-TGI-2".

In [61]:
sm_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'c0cfcd84-d9f0-4a8a-9067-49df8e99f664',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c0cfcd84-d9f0-4a8a-9067-49df8e99f664',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Mon, 29 Jan 2024 17:04:44 GMT'},
  'RetryAttempts': 0}}

In [83]:
sm_client.delete_endpoint_config(EndpointConfigName='SOLAR-TGI-2')

{'ResponseMetadata': {'RequestId': '726c1905-cb0e-4dcb-883c-62aa1fb88ca4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '726c1905-cb0e-4dcb-883c-62aa1fb88ca4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Mon, 29 Jan 2024 17:10:07 GMT'},
  'RetryAttempts': 0}}